In [ ]:
!unzip /content/dataset.zip
!unzip /content/NLP-project.zip

Archive:  /content/dataset.zip
   creating: dataset/
   creating: dataset/SemEval2010_task8_scorer-v1.2/
  inflating: dataset/SemEval2010_task8_scorer-v1.2/answer_key1.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/answer_key2.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/answer_key3.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/answer_key5.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/proposed_answer1.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/proposed_answer2.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/proposed_answer3.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/proposed_answer4.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/proposed_answer5.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/README.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/result_scores1.txt  
  inflating: dataset/SemEval2010_task8_scorer-v1.2/result_scores2.txt  
  inflating: dataset/SemEval2010_task8_sc

In [ ]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3 MB 412 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires numpy>=1.17, but you have numpy 1.16.1 which is incompatible.
scikit-image 0.18.3 requires numpy>=1.16.5, but you have numpy 1.16.1 which is incompatible.
pywavelets 1.2.0 requires numpy>=1.17.3, but you have numpy 1.16.1 which is incompatible.
pyerfa 2.0.0.1 requires numpy>=1.17, but you have numpy 1.16.1 which is incompatible.
pyarrow 3.0.0 requires numpy>=1.16.6, but you have numpy 1.16.1 which is incompatible.
kapre 0.3.6 requires numpy>=1.18.5, but you have numpy 1.16.1 which is incompatible.
jaxlib 0.1.71+cuda111 requires numpy>=1.18, but you have numpy 1.16.1 which is incompatibl

## Import needed modules

In [ ]:
import os
os.environ['CLASSPATH'] = "/content/stanford-postagger-2017-06-09"
import collections
import random
import numpy as np
np.random.seed(1337) 
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Concatenate, Embedding, Input, Dropout, Dense, Activation, Bidirectional, GlobalMaxPooling1D
from keras.layers import Convolution1D, LSTM, GRU, CuDNNGRU, CuDNNLSTM, Permute, Lambda, Flatten
from tensorflow.keras import optimizers
from keras.layers import Layer
import tensorflow as tf
from keras import regularizers
from keras.models import load_model
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,confusion_matrix
from nltk.tokenize.stanford import StanfordTokenizer
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

# Dataset stats

In [ ]:
def dataset_stat(filename):
    lines = [line.strip() for line in open(filename)]
    label = []
    for idx in range(0, len(lines), 4):
        if lines[idx+1] == 'Instrument-Agency(e2,e1)' or lines[idx+1] == 'Instrument-Agency(e1,e2)' or lines[idx+1] == 'Member-Collection(e2,e1)' or lines[idx+1] == 'Member-Collection(e1,e2)' or lines[idx+1] == 'Message-Topic(e2,e1)' or lines[idx+1] == 'Message-Topic(e1,e2)' or lines[idx+1] == 'Content-Container(e2,e1)' or lines[idx+1] == 'Content-Container(e1,e2)' or lines[idx+1] == 'Product-Producer(e1,e2)' or lines[idx+1] == 'Product-Producer(e2,e1)' :
                label.append('Other')
        elif lines[idx+1] == 'Component-Whole(e2,e1)' or lines[idx+1] == 'Component-Whole(e1,e2)' : 
            label.append('Component-Whole')
        elif lines[idx+1] == 'Cause-Effect(e2,e1)' or lines[idx+1] == 'Cause-Effect(e1,e2)' :
            label.append('Cause-Effect')
        elif lines[idx+1] == 'Entity-Destination(e2,e1)' or lines[idx+1] == 'Entity-Destination(e1,e2)' :
            label.append('Entity-Destination')
        elif lines[idx+1] == 'Entity-Origin(e1,e2)' or lines[idx+1] == 'Entity-Origin(e2,e1)' :
            label.append('Entity-Origin')
        else:
            label.append(lines[idx+1])

    print(len(lines)//4)
    print(collections.Counter(label))

# Data Preprocessing

In [ ]:
trainFile = '/content/dataset/SemEval2010_task8_training/TRAIN_FILE.TXT'
testFile = '/content/dataset/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT'

op_trainFile = "/content/dataset/train_attn.txt"
op_testFile = "/content/dataset/test_attn.txt"

def createFile(filepath, outputpath):    
    
    def clean_tokens(sent_num, tokens):
        ret = []
        for t in tokens:
            t = t.strip().split()
            if len(t) > 1: 
                print(sent_num, t)
            t = "_".join(t)
            ret.append(t)
        return ret 
    
    fOut = open(outputpath, 'w')
    lines = [line.strip() for line in open(filepath)]
    for idx in range(0, len(lines), 4):
        sentence_num = lines[idx].split("\t")[0]
        sentence = lines[idx].split("\t")[1][1:-1]
        label = lines[idx+1]
        
        if lines[idx+1] == 'Instrument-Agency(e2,e1)' or lines[idx+1] == 'Instrument-Agency(e1,e2)' or lines[idx+1] == 'Member-Collection(e2,e1)' or lines[idx+1] == 'Member-Collection(e1,e2)' or lines[idx+1] == 'Message-Topic(e2,e1)' or lines[idx+1] == 'Message-Topic(e1,e2)' or lines[idx+1] == 'Content-Container(e2,e1)' or lines[idx+1] == 'Content-Container(e1,e2)' or lines[idx+1] == 'Product-Producer(e1,e2)' or lines[idx+1] == 'Product-Producer(e2,e1)' :
            label = 'Other'
        
        sentence = sentence.replace("<e1>", " E1_START ").replace("</e1>", " E1_END ")
        sentence = sentence.replace("<e2>", " E2_START ").replace("</e2>", " E2_END ")

        tokens = StanfordTokenizer().tokenize(sentence)        
        tokens = clean_tokens(sentence_num, tokens)
        
        fOut.write(" ".join([ label, " ".join(tokens) ]))
        fOut.write("\n")
    fOut.close()
        
    print(outputpath, "created")
    

createFile(trainFile, op_trainFile)
createFile(testFile, op_testFile)

Streaming output truncated to the last 5000 lines.
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/

2609 ['1', '1/2']


Streaming output truncated to the last 5000 lines.
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/

7589 ['1', '1/2']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer 

...

10591 ['1', '1/4']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer 

10665 ['6', '1/2']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer 

/content/dataset/test_attn.txt created


In [ ]:
print('Training data statistics')
print(dataset_stat(trainFile))

print('Test data statistics')
print(dataset_stat(testFile))

Training data statistics
8000
Counter({'Other': 4495, 'Cause-Effect': 1003, 'Component-Whole': 941, 'Entity-Destination': 845, 'Entity-Origin': 716})
None
Test data statistics
2717
Counter({'Other': 1527, 'Cause-Effect': 328, 'Component-Whole': 312, 'Entity-Destination': 292, 'Entity-Origin': 258})
None


In [ ]:
train_attn_path = "/content/dataset/train_attn.txt"
test_attn_path = "/content/dataset/test_attn.txt"

train_attn_sp_path = '/content/dataset/train_attn_sp.txt'
val_attn_sp_path = '/content/dataset/val_attn_sp.txt'
test_attn_sp_path = '/content/dataset/test_attn_sp.txt'

train_answer_keys_path = '/content/dataset/train_answer_keys.txt'
val_answer_keys_path = '/content/dataset/val_answer_keys.txt'
test_answer_keys_path = '/content/dataset/test_answer_keys.txt'

In [ ]:
def add_sent_number(file_in, file_out):
    print(file_in)
    print(file_out)
    
    f_in = open(file_in, 'r')
    lines = f_in.readlines()
    f_in.close()
    
    f_out = open(file_out, 'w')
    for i in range(len(lines)):
        num = str(int(i+1))
        ln = num + " " + lines[i]
        f_out.write(ln)
    f_out.close()
    
# Call
add_sent_number(train_attn_path, train_attn_sp_path)
add_sent_number(test_attn_path, test_attn_sp_path)

/content/dataset/train_attn.txt
/content/dataset/train_attn_sp.txt
/content/dataset/test_attn.txt
/content/dataset/test_attn_sp.txt


# Splitting train data into train and validation sets

In [ ]:
def get_val_sent_index(): 
    
    global train_attn_sp_path
    
    label_to_sent_num = {} 

    f_in = open(train_attn_sp_path, 'r')
    lines = f_in.readlines()
    f_in.close()

    for l in lines: 
        l = l.strip().split(" ")[:2]
        num = int(l[0])
        lab = str(l[1])

        if lab not in label_to_sent_num: 
            label_to_sent_num[lab] = [] 

        label_to_sent_num[lab].append(num)


    val_index = []

    for l in label_to_sent_num:
        sent_num = label_to_sent_num[l]
        num = int(len(sent_num) / 10)
        random.shuffle(sent_num)
        random.shuffle(sent_num)
        val_index += sent_num[:num]

    val_index = sorted(val_index)
    print("len(val_index)", len(val_index))
    print("val_index[:5]", val_index[:5])
    return val_index


# Call
val_index = get_val_sent_index()

len(val_index) 796
val_index[:5] [20, 46, 59, 69, 96]


In [ ]:
def train_val_split(val_index):
    global train_attn_sp_path, val_attn_sp_path

    f_in = open(train_attn_sp_path, 'r')
    lines = f_in.readlines()
    f_in.close()

    f_train = open(train_attn_sp_path, 'w')
    f_val = open(val_attn_sp_path, 'w')

    for l in lines: 
        l = l.strip().split(" ")
        num = int(l[0])
        lab = str(l[1])
        
        if num in val_index: 
            f_val.write(" ".join(l) + "\n")
        else: 
            f_train.write(" ".join(l) + "\n")
    
    f_train.close()
    f_val.close()
    
    print("Train - Val - Split ")

# Call
train_val_split(val_index)

Train - Val - Split 


In [ ]:
def train_val_total_check(train_attn_sp_path, val_attn_sp_path):

    def get_count(file_path): 
        print(file_path)

        label_to_sent_count = {} 

        f_in = open(file_path, 'r')
        lines = f_in.readlines()
        f_in.close()

        for l in lines: 
            l = l.strip().split(" ")[:2]
            num = int(l[0])
            lab = str(l[1])

            if lab not in label_to_sent_count: 
                label_to_sent_count[lab] = 0 

            label_to_sent_count[lab] += 1
            
        return label_to_sent_count
    
    train = get_count(train_attn_sp_path)
    val = get_count(val_attn_sp_path)
    
    c = 0
    for l in train: 
        c += train[l]
        if l in val:
            c += val[l]
    
    print(c)


# Call
train_val_total_check(train_attn_sp_path, val_attn_sp_path)

/content/dataset/train_attn_sp.txt
/content/dataset/val_attn_sp.txt
8000


In [ ]:
def create_answer_keys(in_file, out_file): 
    
    f_in = open(in_file, 'r')
    lines = f_in.readlines()
    f_in.close()

    f_out = open(out_file, 'w')
    
    for i in range(0, len(lines)):
        l = lines[i].strip().split(" ")
        num = str(i+1)
        lab = str(l[1])
        f_out.write(num + "\t" + lab) 
        f_out.write("\n") 
    f_out.close()
    
    print(out_file + " " + "Created")

create_answer_keys(train_attn_sp_path, train_answer_keys_path)
create_answer_keys(val_attn_sp_path, val_answer_keys_path)
create_answer_keys(test_attn_sp_path, test_answer_keys_path)

/content/dataset/train_answer_keys.txt Created
/content/dataset/val_answer_keys.txt Created
/content/dataset/test_answer_keys.txt Created


# Initializing Word Embeddings and other needed dictionaries 

In [ ]:
emb_google_txt = '/content/word_embeddings/GoogleNews-vectors-negative300_sample.txt'
avg_vec_file = '/content/word_embeddings/GoogleNews-vectors-negative300_avg_vec.txt'

data_all_file_path = "/content/dataset/data_all_files"

files = [train_attn_sp_path, val_attn_sp_path, test_attn_sp_path]

label_to_int = {'Other': 0, 'Entity-Destination(e1,e2)': 1, 'Entity-Destination(e2,e1)': 2, 'Cause-Effect(e1,e2)': 3, 'Cause-Effect(e2,e1)': 4, 'Component-Whole(e1,e2)': 5, 'Component-Whole(e2,e1)': 6, 'Entity-Origin(e1,e2)': 7, 'Entity-Origin(e2,e1)': 8}
int_to_label= {0: 'Other', 1: 'Entity-Destination(e1,e2)', 2: 'Entity-Destination(e2,e1)', 3: 'Cause-Effect(e1,e2)', 4: 'Cause-Effect(e2,e1)', 5: 'Component-Whole(e1,e2)', 6: 'Component-Whole(e2,e1)', 7: 'Entity-Origin(e1,e2)', 8: 'Entity-Origin(e2,e1)'} 

words_dataset = set()

def create_words_dataset():
    for f_name in files: 
        f = open(f_name, 'r')
        lines = f.readlines()
        f.close()
        
        lines = [ l.strip().split(" ")[2:]  for l in lines]
    
        for line in lines:
            for w in line:
                words_dataset.add(w)
    
    print("len(words_dataset)", len(words_dataset))
    
    with open('/content/dataset/words_dataset.txt', 'w') as f: 
        for w in sorted(words_dataset):
            f.write(str(w) + "\n")
        print('/content/dataset/words_dataset.txt created')
    
    
create_words_dataset()

word_to_emb = {}

with open(emb_google_txt, 'r', encoding='utf-8') as f: 
    first = True
    
    for line in f:
        if first == True:
            first = False
            continue
        line = line.strip().split()
        if len(line) != 301:
            continue 
        word = str(line[0])
        vec = [float(x) for x in line[1:]]
        vec = np.array(vec, dtype='float64')
        
        if word in words_dataset:
            word_to_emb[word] = vec
        elif word.lower() in words_dataset and word.lower() not in word_to_emb: 
            word_to_emb[word.lower()] = vec

len(words_dataset) 25655
/content/dataset/words_dataset.txt created


In [ ]:
def get_avg_vec(file_name):
    with open(file_name, 'r') as f:
        line = f.readline()
        line = line.strip().split()
        line = [float(x) for x in line]
        avg_vec = np.array(line, dtype='float64')
        print("avg_vec.shape", avg_vec.shape)
        return avg_vec

avg_vec = get_avg_vec(avg_vec_file)
word_to_int = {}
embedding = []

unknown_words = 0
word_to_int['PADDING'] = len(word_to_int)
embedding.append(np.zeros(300, dtype='float64'))

for w in sorted(words_dataset): 
    word_to_int[w] = len(word_to_int)
    if w in word_to_emb:
        embedding.append(word_to_emb[w])
    elif w.lower() in word_to_emb:
        embedding.append(word_to_emb[w.lower()])
    else:
        unknown_words += 1
        embedding.append(avg_vec)

embedding = np.array(embedding, dtype='float64')
print("len(word_to_int)", len(word_to_int)) # 25656
print("embedding.shape", embedding.shape) # (25656, 300)
print("unknown_words", unknown_words)

avg_vec.shape (300,)
len(word_to_int) 25656
embedding.shape (25656, 300)
unknown_words 25500


In [ ]:
def get_max_sent_len(files):
    max_sent_len = 0 
    for fname in files: 
        f = open(fname, 'r')
        lines = f.readlines()
        f.close()
        for l in lines: 
            l = l.strip().split(" ")[2:]
            max_sent_len = max(max_sent_len, len(l))
    return max_sent_len

max_sent_len = get_max_sent_len(files)
print("max_sent_len:", max_sent_len)

max_sent_len: 101


# Padding

In [ ]:
def create_matrices(file_name, word_to_int, label_to_int, max_sent_len):
    X = []
    Y = []
    
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    lines = [line.strip().split() for line in lines]
    
    for line in lines: 
        Y.append(label_to_int[line[1]])
        line = line[2:]
        tmp = np.zeros(max_sent_len, dtype='int32')
        for i in range(len(line)):
            tmp[i] = word_to_int[line[i]]
        X.append(tmp)

        
    X = np.array(X, dtype='int32')
    Y = np.array(Y, dtype='int32')
    
    return [X, Y]

train_set = create_matrices(train_attn_sp_path, word_to_int, label_to_int, max_sent_len)
val_set = create_matrices(val_attn_sp_path, word_to_int, label_to_int, max_sent_len)
test_set = create_matrices(test_attn_sp_path, word_to_int, label_to_int, max_sent_len)

data_all = [train_set, val_set, test_set, embedding, label_to_int, int_to_label]

np.save(data_all_file_path, data_all)

In [ ]:
data_npy_path = '/content/dataset/data_all_files.npy'
scorer_perl_script = "/content/dataset/SemEval2010_task8_scorer-v1.2/semeval2010_task8_scorer-v1.2.pl"
train_answer_key = "/content/dataset/train_answer_keys.txt"
val_answer_key = "/content/dataset/val_answer_keys.txt"
test_answer_key = "/content/dataset/test_answer_keys.txt"

proposed_ans = "/content/dataset/proposed_ans.txt"
scorer_output = "/content/dataset/scorer_output.txt"
perl_exe_path = "/usr/bin/perl" 

In [ ]:
train_set, val_set, test_set, embedding, label_to_int, int_to_label = np.load(data_npy_path)

train_x, train_y = train_set
val_x, val_y = val_set
test_x, test_y = test_set

train_x_copy = train_x
train_y_copy = train_y

max_sentence_len = train_x.shape[1]
n_out = len(label_to_int)

print("train_x.shape", train_x.shape)
print("train_y.shape", train_y.shape)
print("val_x.shape", val_x.shape)
print("val_y.shape", val_y.shape)
print("test_x.shape", test_x.shape)
print("test_y.shape", test_y.shape)
print("embedding.shape", embedding.shape)
print("len(label_to_int)", len(label_to_int))
print("len(int_to_label)", len(int_to_label))
print("max_sentence_len", max_sentence_len)
print("n_out", n_out)
print()

train_x.shape (7204, 101)
train_y.shape (7204,)
val_x.shape (796, 101)
val_y.shape (796,)
test_x.shape (2717, 101)
test_y.shape (2717,)
embedding.shape (25656, 300)
len(label_to_int) 9
len(int_to_label) 9
max_sentence_len 101
n_out 9



In [ ]:
def get_mask_entities(x):
    ''' 1 for entity words, 0 otherwise '''
    
    ret = np.zeros_like(x)
    #print("get_mask shape", ret.shape)
    # e1s 2409
    # e1e 2408
    # e2s 2411
    # e2e 2410
    for i in range(x.shape[0]): 
        e1 = [0, 0]
        e2 = [0, 0]
        for j in range(x.shape[1]):
            if x[i][j] == 2409:
                e1[0] = j
            elif x[i][j] == 2408: 
                e1[1] = j
            elif x[i][j] == 2411: 
                e2[0] = j
            elif x[i][j] == 2410: 
                e2[1] = j
                break
        for j in range(e1[0]+1, e1[1]): 
            ret[i][j] = 1
        for j in range(e2[0]+1, e2[1]): 
            ret[i][j] = 1
    
    
    return ret

train_x_mask = get_mask_entities(train_x)
train_x_mask_copy = train_x_mask
val_x_mask = get_mask_entities(val_x)
test_x_mask = get_mask_entities(test_x)

print("train_x_mask.shape", train_x_mask.shape)
print("train_x_mask_copy.shape", train_x_mask_copy.shape)
print("val_x_mask.shape", val_x_mask.shape)
print("test_x_mask.shape", test_x_mask.shape)

train_x_mask.shape (7204, 101)
train_x_mask_copy.shape (7204, 101)
val_x_mask.shape (796, 101)
test_x_mask.shape (2717, 101)


# Model

In [ ]:
# Parameters 

units = 64
batch_size = 128 #10
dropout_emb = 0.64
dropout_model = 0.32
dropout_pen = 0.32
l2_val = 0.00001
learning_rate = 1.0
activation_fn = 'tanh'
nb_epoch = 256
adadelta = optimizers.Adadelta(lr=learning_rate, decay=0.0)
save_model = True
es_epoch_stop = 20

print("units", units)
print("batch_size", batch_size)
print("dropout_emb", dropout_emb)
print("dropout_model", dropout_model)
print("dropout_pen", dropout_pen)
print("l2_val", l2_val)
print("learning_rate", learning_rate)
print("activation_fn", activation_fn)
print("nb_epoch", nb_epoch)
print("adadelta", adadelta)
print("save_model", save_model)
print("es_epoch_stop", es_epoch_stop)
print()

embeddings_reg = regularizers.l2(l2_val)
kernel_reg = regularizers.l2(l2_val)
recurrent_reg = regularizers.l2(l2_val)

class MaskMaxPoolingLayer(Layer):
    # input_shape [(None, 101, 600), (None, 101)]
    # input:  (?, 101, 600) (?, 101)
    # output: (?, 600)
    
    def __init__(self, **kwargs):
        super(MaskMaxPoolingLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(MaskMaxPoolingLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        x_1_float32 = K.cast(x[1], dtype='float32')
        x_0 = K.permute_dimensions(x[0], pattern=[2, 0, 1])
        x_0 = tf.multiply(x_0, x_1_float32)
        x_0 = K.permute_dimensions(x_0, pattern=[1, 2, 0])
        x_0 = K.max(x_0, axis=-2)
        return x_0
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][-1])

units 64
batch_size 128
dropout_emb 0.64
dropout_model 0.32
dropout_pen 0.32
l2_val 1e-05
learning_rate 1.0
activation_fn tanh
nb_epoch 256
adadelta <keras.optimizer_v2.adadelta.Adadelta object at 0x7f61a0aa4cd0>
save_model True
es_epoch_stop 20



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)


In [ ]:
# Model

words_input = Input(shape=(max_sentence_len,), dtype='int32', name='words_input')
words_input_mask = Input(shape=(max_sentence_len,), dtype='int32', name='words_input_mask')

words = Embedding(embedding.shape[0], embedding.shape[1], weights=[embedding], trainable=True, embeddings_regularizer=embeddings_reg, name="words_Embedding")(words_input)
words = Dropout(dropout_emb)(words)

output = Convolution1D(filters=256, kernel_size=3, activation=activation_fn, padding='same', strides=1, kernel_regularizer=kernel_reg)(words)
output = Dropout(dropout_model)(output)

output = Bidirectional( CuDNNLSTM(units, return_sequences=True, recurrent_regularizer=recurrent_reg), merge_mode='concat') (output)
output_h = Activation('tanh') (output)

output1 = GlobalMaxPooling1D()(output_h) 
output2 = MaskMaxPoolingLayer()([output_h, words_input_mask]) 

output = Dense(1, kernel_regularizer=kernel_reg)(output_h)
output = Permute((2, 1))(output)
output = Activation('softmax', name="attn_softmax")(output)
output = Lambda(lambda x: tf.matmul(x[0], x[1])) ([output, output_h])
output3 = Flatten() (output)

output = Concatenate()([output1, output2, output3])
output = Dropout(dropout_pen)(output)

output = Dense(300, kernel_regularizer=kernel_reg, activation='relu')(output)
output = Dense(n_out, kernel_regularizer=kernel_reg)(output)
output = Activation('softmax')(output)

model = Model(inputs=[words_input, words_input_mask], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer=adadelta, metrics=['accuracy'])
model.summary(line_length=120)

Model: "model"
________________________________________________________________________________________________________________________
 Layer (type)                          Output Shape               Param #       Connected to                            
 words_input (InputLayer)              [(None, 101)]              0             []                                      
                                                                                                                        
 words_Embedding (Embedding)           (None, 101, 300)           7696800       ['words_input[0][0]']                   
                                                                                                                        
 dropout (Dropout)                     (None, 101, 300)           0             ['words_Embedding[0][0]']               
                                                                                                                        
 conv1d (Conv1D) 

In [ ]:
def get_precision(test_y_pred, test_y, label): 
    label_count = 0
    label_count_correct = 0
    
    for i in range(len(test_y_pred)):
        if test_y_pred[i] == label: 
            label_count += 1 
            if test_y_pred[i] == test_y[i]: 
                label_count_correct += 1
    
    if label_count_correct == 0: 
        return 0.0
    else: 
        ret = float(label_count_correct) / float(label_count)
        return ret
        

def get_macro_f1(test_y_pred, test_y, n_out):
    f1_sum = 0
    f1_count = 0
    for label in range(1, n_out):        
        prec = get_precision(test_y_pred, test_y, label)
        recall = get_precision(test_y, test_y_pred, label)
        f1 = 0 if float(prec+recall)==float(0) else float(2*prec*recall/float(prec+recall))
        f1_sum += f1
        f1_count += 1
    macro_f1 = float(f1_sum) / float(f1_count)    
    return macro_f1
    

def get_accuracy(test_y_pred, test_y):
    acc =  float(np.sum(test_y_pred == test_y)) / float(len(test_y))
    return acc
    
    
def predict_classes(prediction):
    return prediction.argmax(axis=-1)

def get_PRF1_semeval(y_pred, answer_key):
    
    f_out = open(proposed_ans, 'w')
    for i in range(len(y_pred)):
        f_out.write(str(i+1) + "\t" + int_to_label[y_pred[i]] + "\n" )
    f_out.close()
    
    if os.name == 'nt': 
        os.system(perl_exe_path + " " + scorer_perl_script + " " + proposed_ans + " " + answer_key + " > " + scorer_output)
    else: 
        os.system("perl " + " " + scorer_perl_script + " " + proposed_ans + " " + answer_key + " > " + scorer_output)
        
    f_in = open(scorer_output, 'r')
    lines = f_in.readlines()
    f_in.close()
    
    lines = [l  for l in lines[-30:] if l.strip() != '']
    PRF1 = lines[-2].strip().split()
    acc = float(lines[10].strip().split()[-1][:-1]) / 100.0
    P = float(PRF1[2][:-1]) / 100.0
    R = float(PRF1[5][:-1]) / 100.0
    macro_f1 = float(PRF1[8][:-1]) / 100.0
    
    return (acc, P, R, macro_f1)

# Training

In [ ]:
print("Start training... \n")

train_max_acc = 0
train_max_f1 = 0
val_max_acc = 0
val_max_f1 = 0
test_max_acc = 0
test_max_f1 = 0

test_f1_final = 0
test_f1_final_max = 0

es_epoch = 0 

for epoch in range(nb_epoch):
    print("Epoch: ", epoch+1, "/", nb_epoch)
    es_epoch += 1
    if es_epoch > es_epoch_stop:
        print("Early Stopping...") 
        break
    
    index = np.arange(len(train_x))
    np.random.shuffle(index)
    train_x = train_x[index]
    train_x_mask = train_x_mask[index]
    train_y = train_y[index]
     
    # Input lists for fit and predict
    train_input_list = [train_x, train_x_mask]
    train_input_list_copy = [train_x_copy, train_x_mask_copy]
    val_input_list = [val_x, val_x_mask]
    test_input_list = [test_x, test_x_mask]

    model.fit(train_input_list, train_y, batch_size=batch_size, verbose=1, epochs=1)
    
    train_y_pred = predict_classes(model.predict(train_input_list_copy, verbose=1))
    val_y_pred = predict_classes(model.predict(val_input_list, verbose=1))
    
    
    train_PRF1 = get_PRF1_semeval(train_y_pred, train_answer_key)
    val_PRF1 = get_PRF1_semeval(val_y_pred, val_answer_key)
  

    train_max_f1 = max(train_max_f1, train_PRF1[3])
    val_max_f1 = max(val_max_f1, val_PRF1[3])
  

    train_max_acc = max(train_max_acc, train_PRF1[0])
    val_max_acc = max(val_max_acc, val_PRF1[0])

    
    if val_max_f1 == val_PRF1[3]: 
        if save_model : 
            model_file_name = "model.keras"
            model.save('/content/dataset/' + model_file_name)
            print("Model saved", model_file_name)
        es_epoch = 0


    print("Val   Accuracy: %.4f " % (val_PRF1[0]))
    
    print("Train Macro F1 Semeval Official: %.4f " % (train_PRF1[3]))
    print("Val   Macro F1 Semeval Official: %.4f " % (val_PRF1[3]))

        
    print()
    

Start training... 

Epoch:  1 / 256
25/25 [==============================] - 0s 19ms/step
Model saved model.keras
Val   Accuracy: 0.5641 
Train Macro F1 Semeval Official: 0.0000 
Val   Macro F1 Semeval Official: 0.0000 

Epoch:  2 / 256
25/25 [==============================] - 0s 15ms/step
Model saved model.keras
Val   Accuracy: 0.6030 
Train Macro F1 Semeval Official: 0.1625 
Val   Macro F1 Semeval Official: 0.1623 

Epoch:  3 / 256
25/25 [==============================] - 0s 15ms/step
Model saved model.keras
Val   Accuracy: 0.6193 
Train Macro F1 Semeval Official: 0.2915 
Val   Macro F1 Semeval Official: 0.2651 

Epoch:  4 / 256
25/25 [==============================] - 0s 15ms/step
Model saved model.keras
Val   Accuracy: 0.6897 
Train Macro F1 Semeval Official: 0.4655 
Val   Macro F1 Semeval Official: 0.4597 

Epoch:  5 / 256
25/25 [==============================] - 0s 15ms/step
Model saved model.keras
Val   Accuracy: 0.6859 
Train Macro F1 Semeval Official: 0.5018 
Val   Macro F1 Se

# Model testing

In [ ]:
model = load_model('/content/dataset/model.keras',custom_objects={'MaskMaxPoolingLayer': MaskMaxPoolingLayer})
test_input_list = [test_x, test_x_mask]
test_y_pred = predict_classes(model.predict(test_input_list, verbose=1))
test_PRF1 = get_PRF1_semeval(test_y_pred, test_answer_key)

print("Test Accuracy: %.4f " % (test_PRF1[0]))
print("Test Precision %.4f | Recall %.4f | macro_F1: %.4f" % test_PRF1[1:] )

85/85 [==============================] - 11s 21ms/step
Test Accuracy: 0.8495 
Test Precision 0.8026 | Recall 0.8027 | macro_F1: 0.8016


# Predictions on sentences

In [ ]:
def sent_predict(sent, flag = 1):
    test_sent = sent.replace("<e1>", " E1_START ").replace("</e1>", " E1_END ")
    test_sent = test_sent.replace("<e2>", " E2_START ").replace("</e2>", " E2_END ")
    toke = StanfordTokenizer().tokenize(test_sent)
    ret = []
    for t in toke:
        t = t.strip().split()
        if len(t) > 1:
            print(t)
        t = "_".join(t)
        ret.append(t)


    test = []
    tmp = np.zeros(max_sent_len, dtype='int32')
    for i in range(len(ret)):
        tmp[i] = word_to_int[ret[i]]
        test.append(tmp)

         
    test = np.array(test, dtype='int32')
    test_list = [test, get_mask_entities(test)]
    pred = np.argmax(model.predict(test_list, verbose=1))
    # actual = label_to_int[true_label]
    print('\n')
    print(int_to_label[pred])
    # print('model predicted the correct output')
    '''
    elif pred != actual and flag == 0:
      print('\n')
      print('model failed to predict this particular sentence, needs improving!!')
      print(sent)
      print('actual label is :'+true_label)
      print('But, the model predicted it as :'+int_to_label[pred])
      print('\n')
    
    # uncomment the below cell if predicting on a file to measure the performance of the trained model 
    return pred, actual 
    '''

In [ ]:
# test2 = 'Ten million quake <e1>survivors</e1> moved into makeshift <e2>houses</e2>.' # Entity-Destination(e1,e2)

test1 = 'As a <e1>landscape</e1> <e2>company</e2> in Atlanta, we know which plants thrive in this planting zone and know the optimum landscaping designs for local yards and business' # other
test0 = 'Avian <e1>influenza</e1> is an infectious disease of birds caused by type A strains of the influenza <e2>virus</e2>.' # cause-effect(e2,e1)

test = 'The current <e1>band</e1> arrived from the Lafayette <e2>competition</e2>.' 
sent_predict(test, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


1/1 [==============================] - 0s 77ms/step


Entity-Origin(e1,e2)


In [ ]:
f = open('/content/random_50_sent.txt', 'r')
lines = f.readlines()

evallist = []
labels = []

for line in lines:
    label = line.strip().split()[0]
    a = line.strip().split()[1:]
    sent = ' '.join(a)
    pred, tlabel = sent_predict(sent, label, 0)
    evallist.append(pred)
    labels.append(tlabel)

labels = np.array(labels, dtype=np.float32)
evallist = np.array(evallist, dtype=np.float32)

print("\nTest Accuracy: %.4f " % (accuracy_score(labels, evallist)))
print("Test Precision: %.4f " % (precision_score(labels, evallist,
                                           average='macro')))
print("Test Recall: %.4f " % (recall_score(labels, evallist,
                                           average='macro')))
print("Test F1: %.4f " % (f1_score(labels, evallist,
                                           average='macro')))
confusion_matrix(labels, evallist)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


1/1 [==============================] - 0s 34ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


1/1 [==============================] - 0s 78ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


1/1 [==============================] - 0s 36ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.parse.corenlp.CoreNLPTokenizer instead.'
  after removing the cwd from sys.path.


1/1 [==============================] - 0s 38ms/step

Test Accuracy: 0.8800 
Test Precision: 0.8700 
Test Recall: 0.8889 
Test F1: 0.8783 


array([[24,  1,  0,  1,  0,  1,  0],
       [ 1,  5,  0,  0,  0,  0,  0],
       [ 0,  0,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  3,  0,  0,  1],
       [ 0,  0,  0,  0,  2,  0,  0],
       [ 1,  0,  0,  0,  0,  3,  0],
       [ 0,  0,  0,  0,  0,  0,  5]])